In [2]:
import math
import numpy as np
from scipy.stats import spearmanr, pearsonr  

In [3]:

systems = ["TrainTicket", "MicroserviceConsul", "TMS2020"]

coupling    = {"TrainTicket": 8.5, "MicroserviceConsul": 10.0, "TMS2020": 6.1}
granularity = {"TrainTicket": 0.3, "MicroserviceConsul": 7.2, "TMS2020": 8.9}
cohesion    = {"TrainTicket": 1.2, "MicroserviceConsul": 9.3, "TMS2020": 4.8}


chatty      = {"TrainTicket": 18, "MicroserviceConsul": 0, "TMS2020": 1}
overloaded  = {"TrainTicket": 13, "MicroserviceConsul": 0, "TMS2020": 2}
control     = {"TrainTicket":  6, "MicroserviceConsul": 0, "TMS2020": 1}
intimacy    = {"TrainTicket":  8, "MicroserviceConsul": 0, "TMS2020": 1}


n_services  = {"TrainTicket": 42, "MicroserviceConsul": 3, "TMS2020": 4}

In [4]:
def vec(d): 
    return np.array([d[s] for s in systems], dtype=float)

def log1p_rate(counts):    
    return np.log1p(vec({k: counts[k] / max(n_services[k], 1) for k in systems}))

metrics = {
    "Granularity": vec(granularity),
    "Coupling":    vec(coupling),
    "Cohesion":    vec(cohesion),
}

smells= {
    "Chatty":     log1p_rate(chatty),
    "Overloaded": log1p_rate(overloaded),
    "Control":    log1p_rate(control),
    "Intimacy":   log1p_rate(intimacy),
}


rows = []
for m_name, m_vals in metrics.items():
    row_s, row_p = {"Metric": m_name}, {"Metric": m_name}
    for s_name, s_vals in smells.items():
        rho, _ = spearmanr(m_vals, s_vals)
        r,   _ = pearsonr(m_vals, s_vals)
        row_s[s_name] = round(rho, 3)
        row_p[s_name] = round(r, 3)
    rows.append((row_s, row_p))

print("Spearman ρ (metrics × smells):")
for r,_ in rows: print(r)
print("\nPearson r (metrics × smells):")
for _,r in rows: print(r)


Spearman ρ (metrics × smells):
{'Metric': 'Granularity', 'Chatty': -0.5, 'Overloaded': 0.5, 'Control': 0.5, 'Intimacy': 0.5}
{'Metric': 'Coupling', 'Chatty': -0.5, 'Overloaded': -1.0, 'Control': -1.0, 'Intimacy': -1.0}
{'Metric': 'Cohesion', 'Chatty': -1.0, 'Overloaded': -0.5, 'Control': -0.5, 'Intimacy': -0.5}

Pearson r (metrics × smells):
{'Metric': 'Granularity', 'Chatty': -0.656, 'Overloaded': -0.001, 'Control': 0.075, 'Intimacy': -0.126}
{'Metric': 'Coupling', 'Chatty': -0.51, 'Overloaded': -0.949, 'Control': -0.97, 'Intimacy': -0.902}
{'Metric': 'Cohesion', 'Chatty': -0.997, 'Overloaded': -0.7, 'Control': -0.645, 'Intimacy': -0.784}
